In [1]:
import h5py
from ipywidgets import IntSlider, interact, Dropdown
import numpy as np
import matplotlib.pylab as plt

from spikelib.fitting import fit_temp_sta
from spikelib.utils import check_groups, datasets_to_array
%matplotlib inline

# General information about input output

In [60]:
# Data files
intensities = ['nd2-255', 'nd3-255', 'nd4-255', 'nd5-255']
intensity = 'nd3-255'
print(intensity)
path_file = '../../data/processed_protocols/MR-0092t2_modified_analysis_of_protocols_150um_merge.hdf5'

# Groups for raw data in hdf5 file
temp_group = '/sta/{}/temporal/raw/'.format(intensity)
spatial_group = '/sta/{}/spatial/char/'.format(intensity)
valid = '/sta/{}/valid/'.format(intensity)

# Groups for fit data in hdf5 file
temp_fit_group = '/sta/{}/temporal/fit/'.format(intensity)
temp_pars_group = '/sta/{}/temporal/fit_params/'.format(intensity)

nd3-255


# Load Data

In [61]:
# Load data

with h5py.File(path_file, 'r') as stafile:
    print(stafile[spatial_group].attrs.items())
    time_raw = stafile[temp_group].attrs['time']
    time_fit = stafile[temp_fit_group].attrs['time']
    nraw_samples = time_raw.size
    nfit_samples = time_fit.size
    raw_dtype = np.dtype([
        ('id', 'U18'),
        ('temp_raw', 'f8',(nraw_samples,)),
        ('snr', 'f8')
    ])
    fit_dtype = np.dtype([
        ('id', 'U18'),
        ('parameters', 'f8',(5,)),
        ('temp_fit', 'f8', (nfit_samples,)),
    ])
    temporal_sta = np.zeros(len(stafile[temp_group]), dtype=raw_dtype)
    temp_fit = np.zeros((len(temporal_sta)), dtype=fit_dtype)

    for kidx, kkey in enumerate(stafile[temp_group]):
        temporal_sta[kidx]['id'] = kkey
        temporal_sta[kidx]['temp_raw'] = stafile[temp_group][kkey][...]
        temporal_sta[kidx]['snr'] = stafile[spatial_group][kkey][-2]
        temp_fit[kidx]['id'] = kkey
        temp_fit[kidx]['parameters'] = stafile[temp_pars_group][kkey][...]
        temp_fit[kidx]['temp_fit'] = stafile[temp_fit_group][kkey][...]

[(u'col_name', u'angle,semia,semib,x0,y0,snr,frame_max')]


# Visualization of fitting

In [62]:
thr_snr = 4
no_valid_by_snr = np.where(temporal_sta['snr']>thr_snr)[0]

In [63]:
def plot_fit(kidx):
    temp_sta = temporal_sta[kidx]['temp_raw']
    temp_sta_fit = temp_fit[kidx]['temp_fit']
    plt.figure(figsize=(8,6))
    plt.plot(time_raw,temp_sta, 'k+')
    plt.plot(time_raw,temp_sta, 'k')
    plt.plot(time_fit, temp_sta_fit, 'r')
    plt.ylim([-1,1])
    plt.xlabel(temporal_sta[kidx]['id'] + '  ({:2.2f})'.format(temporal_sta[kidx]['snr']))
    plt.grid(b=True, which='major', color='k', linestyle='-',alpha=0.2) 
    plt.grid(b=True, which='minor', color='k', linestyle='-',alpha=0.1)
    plt.minorticks_on()
    plt.show()
    plt.close()

In [64]:
# interact(plot_fit, kidx=IntSlider(min=0, max=len(temporal_sta)-1, step=1, value=443));
# interact(plot_fit, kidx=Dropdown(options=no_valid_by_snr));
interact(plot_fit, kidx=Dropdown(options=range(len(temporal_sta)-1)));

aW50ZXJhY3RpdmUoY2hpbGRyZW49KERyb3Bkb3duKGRlc2NyaXB0aW9uPXUna2lkeCcsIG9wdGlvbnM9KDAsIDEsIDIsIDMsIDQsIDUsIDYsIDcsIDgsIDksIDEwLCAxMSwgMTIsIDEzLCAxNCzigKY=


In [70]:
np.where(temporal_sta[:]['id'] == 'temp_40')[0][0]

335

# Fitting for specific unit


In [71]:
kidx = 335
kcell = temporal_sta[kidx]
temp_to_fit = kcell['temp_raw']

params, tmp_fit = fit_temp_sta(temp_to_fit, time_raw, time_fit, 
                               tau1=-0.15, tau2=-0.06, amp1=0.01, amp2=0.2, min_time=None,
                               max_time=None)

temp_fit[kidx]['parameters'] = np.asarray(params)
temp_fit[kidx]['temp_fit'] = tmp_fit
params

Parameters([('amp1', <Parameter 'amp1', 0.011562347270259732, bounds=[-1:1]>),
            ('amp2', <Parameter 'amp2', 0.2032449083210508, bounds=[-1:1]>),
            ('tau1',
             <Parameter 'tau1', -0.16842976642617924, bounds=[-0.4685550999043143:-0.016734110710868368]>),
            ('tau2',
             <Parameter 'tau2', -0.05970604654206568, bounds=[-0.4685550999043143:-0.016734110710868368]>),
            ('n', <Parameter 'n', 19.25172178278, bounds=[-inf:20]>)])

In [69]:
kidx = 160
temp_fit[kidx]['parameters'] *= 0.
temp_fit[kidx]['temp_fit'] *= 0.

In [ ]:
no_valid = []

In [ ]:
for kidx in no_valid:
    temp_fit[kidx]['parameters'] *= 0.
    temp_fit[kidx]['temp_fit'] *= 0.

## Save to hdf5

In [72]:
with h5py.File(path_file, 'r+') as fanalysis:
    check_groups(fanalysis, [temp_fit_group, temp_pars_group])      
    for kunit in temp_fit:
        key = kunit['id']
        fanalysis[temp_fit_group+key][...] = kunit['temp_fit']
        fanalysis[temp_pars_group+key][...] = kunit['parameters']